## 付録D　電気ヒートポンプ温水暖房機（フロン系）

### D.1 記号及び単位

省略

### D.2 エネルギー消費量

#### D.2.1 消費電力量

$$\large
E_{E,hs} = \frac{Q_{out,H,hs}}{e_{hs}} \times 10^{3} \div 3600
$$

$E_{E,hs}$：1 時間当たりの温水暖房用熱源機の消費電力量(kWh/h)  
$Q_{out,H,hs}$：1 時間当たりの温水暖房用熱源機の暖房出力(MJ/h)  
$e_{hs}$：1 時間平均の温水暖房用熱源機の効率

$$\large
e_{hs} = \frac{Q_{max,H,hs} \times 10^{6} \div 3600}{P_{max,hs}} \times er_{hs}
$$

$Q_{max,H,hs}$：1 時間当たりの温水暖房用熱源機の暖房出力(MJ/h)  
$P_{max,hs}$：温水暖房用熱源機の最大消費電力(W)  
$er_{hs}$：1 時間平均の温水暖房用熱源機の効率比

$$\large
P_{max,hs} = \frac{P_{rtd,hs}}{0.6}
$$

$P_{rtd,hs}$：温水暖房用熱源機の定格消費電力(W)  

$$\large
P_{rtd,hs} = \frac{q_{rtd,hs}}{e_{rtd}}
$$

$q_{rtd,hs}$：温水暖房用熱源機の定格能力(W)  
$e_{rtd}$：温水暖房用熱源機の定格効率

温水暖房用熱源機の定格効率$e_{rtd}$は4.05とする。

$$\large
q_{rtd,hs} = q_{max,hs} \times 0.8
$$

$q_{max,hs}$：温水暖房用熱源機の最大能力(W)  

$$\large
er_{hs} = \big(1.120656 - 0.03703 \times (\theta_{SW,hs} - \theta_{ex})\big) \times \bigg(1 - \frac{Q_{out,H,hs}}{Q_{max,H,hs}}\bigg)^{2}\\
\large + \big(-0.36786 + 0.012152 \times (\theta_{SW,hs} - \theta_{ex})\big) \times \bigg(1 - \frac{Q_{out,H,hs}}{Q_{max,H,hs}}\bigg)\\
\large + 1
$$

$\theta_{SW,hs}$：1 時間平均の温水暖房用熱源機の往き温水温度(℃)  
$\theta_{ex}$：1 時間平均の外気温度(℃)

#### D.2.2 ガス消費量

1 時間当たりの温水暖房用熱源機のガス消費量$E_{G,hs}$は0とする。

#### D.2.3 石油消費量

1 時間当たりの温水暖房用熱源機の石油消費量$E_{K,hs}$は0とする。

#### D.2.4 その他の一次エネルギー消費量

1 時間当たりの温水暖房用熱源機のその他の一次エネルギー消費量$E_{M,hs}$は0とする。

### D.3 最大暖房出力

$$\large
Q_{max,H,hs} = (11.62 + 0.2781 \times \theta_{ex} - 0.00113 \times \theta_{ex}^{2}\\
\large -0.1271 \times \theta_{SW,hs} - 0.00363 \times \theta_{ex} \times \theta_{SW,hs})
\times \frac{q_{rtd,hs}}{6} \times \frac{C_{def}}{0.85} \times 3600 \times 10^{-6}
$$

$C_{def}$：1 時間あたりのデフロスト補正係数

1 時間あたりのデフロスト補正係数$C_{def}$は、1 時間平均の外気温度$\theta_{ex}$が5℃ 未満かつ 1 時間平均の外気相対湿度$h_{ex}$が80%以上の場合は0.85とし、それ以外の場合は1.0とする。 